In [14]:
using JuMP
using Gurobi
model = Model(Gurobi.Optimizer)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-04


A JuMP Model
├ solver: Gurobi
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [15]:
# Import the necessary libraries
using HTTP
using JSON

# Store the URL
url = "https://fireroad.mit.edu/requirements/get_json/major2A/"

# Send the request and get the response
reqs = HTTP.get(url)

# Parse the JSON response
datafile = JSON.parse(String(reqs.body))

Dict{String, Any} with 7 entries:
  "short-title"     => "2"
  "medium-title"    => "2-A Major"
  "title-no-degree" => "Mechanical Engineering"
  "reqs"            => Any[Dict{String, Any}("connection-type"=>"all", "thresho…
  "list-id"         => "major2a.reql"
  "title"           => "Engineering as Recommended by the Department of Mechani…
  "desc"            => "Course 2-A is designed for students whose academic and …

In [16]:
# Create a mapping for comparison types
type_mapping = Dict("LT" => "lt", "GT" => "gt", "GTE" => "geq", "LTE" => "leq")

function create_requirements(data, connector=false, cutoff=1, ineq="geq")
    mandatory = []
    all_reqs = []
    together = false
    comb_req = []
    
    for section in data["reqs"]
        ineq = "geq"
        cutoff = 1
        
        # Base case: if there's a "req" key in the section
        if haskey(section, "req")
            if connector
                push!(mandatory, section["req"])
                continue
            else
                push!(all_reqs, section["req"])
                continue
            end
        end

        # Recursive case: checking for special thresholds
        if haskey(section, "distinct-threshold")
            cutoff = section["distinct-threshold"]["cutoff"]
            ineq = type_mapping[section["distinct-threshold"]["type"]]
            if haskey(section, "threshold") && section["threshold"]["cutoff"] != cutoff
                together = true
                total_cutoff = section["threshold"]["cutoff"]
                total_ineq = type_mapping[section["threshold"]["type"]]
            end
        elseif haskey(section, "threshold")
            cutoff = section["threshold"]["cutoff"]
            ineq = type_mapping[section["threshold"]["type"]]
        end
        
        # Building based on connection types
        if section["connection-type"] == "all"
            next = create_requirements(section, true)
            if together
                comb_req = vcat(comb_req, reduce(vcat, getindex.(next, 1)))
            else
                append!(all_reqs, next)
            end
        elseif section["connection-type"] == "any"
            next = create_requirements(section)
            if together
                comb_req = vcat(comb_req, reduce(vcat, getindex.(next, 1)))
            else
                one_req = (next, ineq, cutoff)
                push!(all_reqs, one_req)
            end
        else
            return create_requirements(section)
        end
        
        if together
            push!(all_reqs, (comb_req, total_ineq, total_cutoff))
        end
    end

    if !isempty(mandatory)
        push!(all_reqs, (mandatory, "eq", length(mandatory)))
    end
    
    return all_reqs
end

# Call the function
requirements = create_requirements(datafile)

6-element Vector{Any}:
 (Any["2.00", "2.00A", "2.00B", "2.00C"], "geq", 1)
 (Any["2.678", "2.674"], "geq", 1)
 (Any["2.009", "2.013", "2.750", "2.760"], "geq", 1)
 (Any["2.001", "2.003", "2.005", "2.086", "2.671", "18.03"], "eq", 6)
 "72 units"
 (Any["2.671", "2.009", "2.013", "2.750", "2.760"], "geq", 1)

In [17]:
function create_inequality_constraint(model, expr, ineq::String, value)
    if ineq == "geq"
        @constraint(model, expr >= value)
    elseif ineq == "eq"
        @constraint(model, expr == value)
    elseif ineq == "leq"
        @constraint(model, expr <= value)
    else
        error("Unknown inequality type: $ineq")
    end
end


function add_requirements_constraints(model::Model, requirements::Vector, course_vars::Dict{String, JuMP.VariableRef})
    for req in requirements
        courses, ineq, cutoff = req

        # If the course requirement is a nested structure (i.e., a vector of other requirements)
        if isa(courses, Vector{Any}) && isa(courses[1], Tuple)
            # Recursively handle the nested requirements
            for nested_req in courses
                add_requirements_constraints(model, [nested_req], course_vars)
            end
        else
            # Retrieve the variables for each course in the list
            course_vars_expr = sum(get(course_vars, course, 0) for course in courses)

            # Create the appropriate inequality constraint
            create_inequality_constraint(model, course_vars_expr, ineq, cutoff)
        end
    end
end

add_requirements_constraints (generic function with 1 method)